In [84]:
#!pip install -U selenium
#!pip install maya

In [195]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import numpy as np
import pickle
import time
from datetime import datetime
import maya
import re
import os
import json
import pickle
from glob import glob

In [174]:
# get a reference to the download menu. This will run before the page has 
# finished loading, so we stick it in a while loop and just keep looping
# until we're successful.
def find_by_id(element_id):
    while True:
        try:
            target = driver.find_element_by_id(element_id)
        except NoSuchElementException:
            time.sleep(0.2)
            continue
        else:
            break
    return target

def find_by_class(element_class):
    counter_broke = 0
    while True:
        try:
            target = driver.find_element_by_class_name(element_class)
        except NoSuchElementException:
            time.sleep(0.2)
            if (counter_broke > 1):
                break
            counter_broke += 1
            continue
        else:
            break
    return target

def get_link(link):
    driver.implicitly_wait(2.5)
    try:
        driver.get(link)
    except TimeoutException:
        print("problem loading: " + link)
        driver.refresh()
            

In [175]:
with open("dep_list.txt", "rb") as fp:   # Unpickling
    dep_list = pickle.load(fp)
print(dep_list[5000:5100])

['eo-learn-features', 'tf-conversions', 'nmslib', 'pelican-render-math', 'pymarkovchain', 'asana', 'beholder', 'percy', 'ghp-import2', 'pyzbar', 'elfi', 'azureml-model-management-sdk', 'jep', 'mimeparse', 'rqt-dep', 'gprof2dot', 'bmcs', 'web3', 'nbpdfexport', 'mirapy', 'cleo', 'cltk', 'wordninja', 'cassandra-driver', 'planar', 'baobao', 'pykneer', 'mkl_fft', 'fog', 'pyformulas', 'crdp', 'nptdms', 'pysf', 'rx', 'matplotlib-base', 'multiprocessing-on-dill', 'featuretools_tsfresh_primitives', 'shutilwhich', 'chart_studio.plotly', 'postcodes-io-api', 'roscreate', 'audio.coders', 'static3', 'pygmsh', 'platypus-opt', 'tvb-library', 'azure-mgmt-security', 'cn2an', 'docrepr', 'simple-salesforce', 'fusepy', 'orange3-associate', 'pyats.examples', 'rqt-graph', 'annotationframeworkclient', 'python-pam', 'pymssql', 'ml-insights', 'fast_pagerank', 'efficientnet-pytorch', 'apng', 'optimparallel', 'rasa-sdk', 'tf-nightly-gpu-2.0-preview', 'roslib', 'azure-mgmt-netapp', 'django-class', 'python-redis-ca

In [177]:
len(dep_list)

5190

In [178]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
#options.add_argument('--headless') -- using headless mode currently leads to NoSuchElementException 
driver = webdriver.Chrome('/Users/kevingao/Documents/beaut/chromedriver', options=options)

In [179]:
def split_repo_stat(s):
    return int(s.split(" ")[-1].replace(",", ""))

def page_scrape(pack_df, name):
    
    
    pack_row = {}
   
    pack_row["library"] = name

    
    # LIBRARIES.IO
    
    # Stats
    link = "https://libraries.io/pypi/" + name
    try:
        get_link(link)
        time.sleep(np.random.rand() * 3 + 2)
        #driver.get(link)  

        page_region = find_by_class("container")
        content_region = page_region.find_elements_by_xpath("//*[@class='col-md-4 sidebar']")

        stats_region = content_region[0]
        sourcerank = stats_region.find_element_by_tag_name("h2")
        pack_row["SourceRank"] = sourcerank.text.split('\n')[1]

        stats = stats_region.find_elements_by_tag_name("dl")[1]
        split_stats = stats.text.split('\n')
        for i in np.arange(len(split_stats) // 2):
            pack_row[stats.text.split('\n')[i * 2]] = stats.text.split('\n')[i * 2 + 1]

        # Versions
        orig_v_link = link + "/versions"
        v_link = orig_v_link
        v_index = 1
        ver_num, ver_date = [], []
        #driver.get(link + "/versions")
        
        while True:
            get_link(v_link)
            time.sleep(np.random.rand() * 3 + 2)
            page_region = find_by_class("col-sm-8")
            try:
                tab_ver = page_region.find_element_by_tag_name("tbody")
            except:
                break
            ind_ver = tab_ver.find_elements_by_tag_name("tr")

            for i in ind_ver:
                front = i.text.split(" Br")[0]
                split_v_d = front.split(" ", 1)
                ver_num.append(split_v_d[0])
                ver_date.append(maya.parse(split_v_d[1]).datetime())
            
            v_index += 1
            v_link = orig_v_link + "?page=" + str(v_index)

        pack_row["Version Number"] = ver_num
        pack_row["Version Time"] = ver_date
    except:
        pass
    
    # PYPI
    link = "https://pypi.org/project/" + name  
    
    try:
        get_link(link)
        time.sleep(np.random.rand() * 3 + 2)
        #driver.get(link)
        page_region = find_by_class("github-repo-info")
        boxes = page_region.find_elements_by_tag_name("li")

        pack_row["stars"] = split_repo_stat(boxes[0].text)
        pack_row["forks"] = split_repo_stat(boxes[1].text)
        pack_row["Open Issues/PRs"] = split_repo_stat(boxes[2].text)
    except:
        pass

    
    # Appending
    pack_df = pack_df.append(pack_row, ignore_index=True)
    
    print(name)
    return pack_df
    
    
'''
package_df = pd.DataFrame()

name = "scipy"
page_scrape(package_df, name)
'''

'\npackage_df = pd.DataFrame()\n\nname = "scipy"\npage_scrape(package_df, name)\n'

In [180]:
start = 0
end = 1000


package_df = pd.DataFrame(columns=["library"])

for i in np.arange(start, end):
    package_df = page_scrape(package_df, dep_list[i])

matplotlib
numpy
pandas
scipy
ipywidgets
seaborn
voila
pillow
scikit-learn
ipython
requests
python-dateutil
packaging
six
pyparsing
jupyter
cycler
kiwisolver
plotly
pytz
sklearn
notebook
fastai
ipykernel
ipython-genutils
traitlets
nbconvert
jinja2
tornado
markupsafe
decorator
jupyter-core
sympy
jsonschema
jupyter-client
pickleshare
pygments
pyzmq
nbformat
tqdm
tensorflow
backcall
wcwidth
prompt-toolkit
parso
pandocfilters
jedi
webencodings
entrypoints
certifi
mistune
testpath
send2trash
xlrd
defusedxml
bleach
attrs
statsmodels
prometheus-client
widgetsnbextension
pyrsistent
terminado
ptyprocess
nbinteract
pexpect
urllib3
chardet
networkx
idna
nltk
jupyterlab
jupyter-console
scikit-image
importlib-metadata
ipympl
folium
qtconsole
keras
lxml
joblib
zipp
beautifulsoup4
jupyter-book
bokeh
bqplot
graphviz
openpyxl
pyyaml
watermark
opencv-python
rise
torch
h5py
click
bs4
cffi
appnope
pycparser
imageio
pytest
geopandas
numba
fastai2
datetime
flask
colorama
altair
torchvision
spacy
qtpy
nbgitp

In [181]:
df = package_df.copy(deep=True)

In [182]:
df

,library,Contributors,Dependencies,Dependent packages,Dependent repositories,First release,Forks,Latest release,Open Issues/PRs,Repository size,SourceRank,Stars,Total releases,Version Number,Version Time,Watchers,forks,stars
0,matplotlib,763,0,10.1K,39.6K,"Jan 9, 2006",5.83K,2 months ago,1663.0,377 MB,28,13.7K,68,"[3.4.2, 3.4.1, 3.4.0, 3.4.0rc3, 3.4.0rc2, 3.4....","[2021-05-08 05:47:00+00:00, 2021-03-31 08:18:0...",569,5874.0,13806.0
1,numpy,736,0,25.5K,84.9K,"Dec 2, 2006",5.62K,about 18 hours ago,2333.0,90.2 MB,29,17.4K,101,"[1.21.0, 1.21.0rc2, 1.21.0rc1, 1.20.3, 1.20.2,...","[2021-06-22 13:39:00+00:00, 2021-06-08 16:38:0...",560,5616.0,17445.0
2,pandas,970,0,14.5K,38.4K,"Dec 25, 2009",12.6K,about 18 hours ago,3669.0,219 MB,28,30.1K,93,"[1.2.5, 1.3.0rc1, 1.2.4, 1.2.3, 1.2.2, 1.2.1, ...","[2021-06-22 13:41:00+00:00, 2021-06-13 19:56:0...","1,113",12560.0,30122.0
3,scipy,680,1,10.4K,43.4K,"Jul 27, 2010",3.76K,3 days ago,1667.0,128 MB,26,8.31K,66,"[1.7.0, 1.7.0rc2, 1.7.0rc1, 1.6.3, 1.6.2, 1.6....","[2021-06-20 16:11:00+00:00, 2021-06-14 15:26:0...",329,3755.0,8312.0
4,ipywidgets,NaN,8,734,7.47K,"Aug 3, 2015",NaN,"Feb 22, 2021",NaN,NaN,17,NaN,123,"[8.0.0a4, 8.0.0a3, 8.0.0a2, 8.0.0a1, 7.6.3, 7....","[2021-02-22 20:59:00+00:00, 2021-02-10 23:51:0...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,imgkit,7,1,11,31,"Jan 30, 2017",62,"Apr 10, 2021",NaN,185 KB,16,628,31,"[1.2.2, 1.2.1, 1.2.0, 1.1.6, 1.1.5, 1.1.4, 1.1...","[2021-04-10 13:49:00+00:00, 2021-04-09 12:28:0...",17,NaN,NaN
996,sphinx_tabs,14,11,41,178,"Jan 22, 2017",43,12 days ago,NaN,1.2 MB,18,157,28,"[3.1.0, 3.0.0, 2.1.0, 2.0.1, 2.0.0, 1.3.0, 1.2...","[2021-06-11 15:04:00+00:00, 2021-05-11 20:38:0...",9,NaN,NaN
997,lea,0,0,0,21,"Jul 27, 2013",3,"Nov 13, 2020",NaN,4.64 MB,9,NaN,50,"[3.4.0, 3.3.2, 3.3.1, 3.3.0, 3.2.3, 3.2.2, 3.2...","[2020-11-13 19:50:00+00:00, 2020-10-14 13:03:0...",5,NaN,NaN
998,requests-html,41,7,135,182,"Feb 24, 2018",823,"Feb 17, 2019",NaN,2.85 MB,18,11.9K,36,"[0.10.0, 0.9.0, 0.8.4, 0.8.3, 0.8.2, 0.8.1, 0....","[2019-02-17 20:14:00+00:00, 2018-03-21 11:50:0...",279,NaN,NaN


In [183]:
package_df[package_df["Repository size"].fillna("0").str.contains("yte")]

,library,Contributors,Dependencies,Dependent packages,Dependent repositories,First release,Forks,Latest release,Open Issues/PRs,Repository size,SourceRank,Stars,Total releases,Version Number,Version Time,Watchers,forks,stars
458,soundfile,1,2,99,370,"Oct 21, 2015",0,"Nov 27, 2019",NaN,1000 Bytes,15,4,7,"[0.10.3.post1, 0.10.2, 0.10.1, 0.9.0.post1, 0....","[2019-11-27 12:16:00+00:00, 2018-04-19 07:13:0...",2,NaN,NaN
556,pprint,1,0,65,346,"Dec 19, 2016",0,"Dec 19, 2016",NaN,0 Bytes,7,0,1,[0.1],[2016-12-19 06:32:00+00:00],0,NaN,NaN


In [184]:
def col2int(s):
    multiplier = 1
    if (type(s) != str):
        return
    s = s.replace(",", "")
    if (s[-1] == "K"):
        multiplier = 1000
        s = s[:-1]
    if (s[-1] == "B" or s[-1] == "s"):
        if (s[-2] == "MB"):
            multiplier = 1
        if (s[-2] == "KB"):
            multiplier = (10 ** -3)
        if (s[-2] == "es"):
            multiplier = (10 ** -6)
        if (s[-2] == "GB"):
            multiplier = (10 ** 3)
        s = s.split(" ")[0]
        
    return float(s) * multiplier

In [185]:
package_col_need = ['Contributors', 'Dependencies', 'Dependent packages',
       'Dependent repositories', 'Forks', 'Repository size', 'SourceRank', 'Stars',
       'Total releases', 'Watchers']

for col in package_col_need:
    package_df[col] = package_df[col].apply(lambda x: col2int(x))

In [186]:
def two_col_combine_not_null(x):
    if x.first_valid_index() is None:
        return None
    else:
        return x[x.first_valid_index()]

In [187]:
package_df["Stars"] = package_df[["stars", "Stars"]].apply(two_col_combine_not_null, axis=1)
package_df["Forks"] = package_df[["forks", "Forks"]].apply(two_col_combine_not_null, axis=1)

In [188]:
package_df = package_df.drop(["stars", "forks"], axis=1)

In [189]:
package_df

,library,Contributors,Dependencies,Dependent packages,Dependent repositories,First release,Forks,Latest release,Open Issues/PRs,Repository size,SourceRank,Stars,Total releases,Version Number,Version Time,Watchers
0,matplotlib,763.0,0.0,10100.0,39600.0,"Jan 9, 2006",5874.0,2 months ago,1663.0,377.00,28.0,13806.0,68.0,"[3.4.2, 3.4.1, 3.4.0, 3.4.0rc3, 3.4.0rc2, 3.4....","[2021-05-08 05:47:00+00:00, 2021-03-31 08:18:0...",569.0
1,numpy,736.0,0.0,25500.0,84900.0,"Dec 2, 2006",5616.0,about 18 hours ago,2333.0,90.20,29.0,17445.0,101.0,"[1.21.0, 1.21.0rc2, 1.21.0rc1, 1.20.3, 1.20.2,...","[2021-06-22 13:39:00+00:00, 2021-06-08 16:38:0...",560.0
2,pandas,970.0,0.0,14500.0,38400.0,"Dec 25, 2009",12560.0,about 18 hours ago,3669.0,219.00,28.0,30122.0,93.0,"[1.2.5, 1.3.0rc1, 1.2.4, 1.2.3, 1.2.2, 1.2.1, ...","[2021-06-22 13:41:00+00:00, 2021-06-13 19:56:0...",1113.0
3,scipy,680.0,1.0,10400.0,43400.0,"Jul 27, 2010",3755.0,3 days ago,1667.0,128.00,26.0,8312.0,66.0,"[1.7.0, 1.7.0rc2, 1.7.0rc1, 1.6.3, 1.6.2, 1.6....","[2021-06-20 16:11:00+00:00, 2021-06-14 15:26:0...",329.0
4,ipywidgets,NaN,8.0,734.0,7470.0,"Aug 3, 2015",NaN,"Feb 22, 2021",NaN,NaN,17.0,NaN,123.0,"[8.0.0a4, 8.0.0a3, 8.0.0a2, 8.0.0a1, 7.6.3, 7....","[2021-02-22 20:59:00+00:00, 2021-02-10 23:51:0...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,imgkit,7.0,1.0,11.0,31.0,"Jan 30, 2017",62.0,"Apr 10, 2021",NaN,185.00,16.0,628.0,31.0,"[1.2.2, 1.2.1, 1.2.0, 1.1.6, 1.1.5, 1.1.4, 1.1...","[2021-04-10 13:49:00+00:00, 2021-04-09 12:28:0...",17.0
996,sphinx_tabs,14.0,11.0,41.0,178.0,"Jan 22, 2017",43.0,12 days ago,NaN,1.20,18.0,157.0,28.0,"[3.1.0, 3.0.0, 2.1.0, 2.0.1, 2.0.0, 1.3.0, 1.2...","[2021-06-11 15:04:00+00:00, 2021-05-11 20:38:0...",9.0
997,lea,0.0,0.0,0.0,21.0,"Jul 27, 2013",3.0,"Nov 13, 2020",NaN,4.64,9.0,NaN,50.0,"[3.4.0, 3.3.2, 3.3.1, 3.3.0, 3.2.3, 3.2.2, 3.2...","[2020-11-13 19:50:00+00:00, 2020-10-14 13:03:0...",5.0
998,requests-html,41.0,7.0,135.0,182.0,"Feb 24, 2018",823.0,"Feb 17, 2019",NaN,2.85,18.0,11900.0,36.0,"[0.10.0, 0.9.0, 0.8.4, 0.8.3, 0.8.2, 0.8.1, 0....","[2019-02-17 20:14:00+00:00, 2018-03-21 11:50:0...",279.0


In [197]:
pack_json = package_df.to_json()
with open("scraper_" + str(start) + "_" + str(end - 1) + ".txt",'w') as f:
    json.dump(pack_json, f)

---------------------

In [262]:
stopper

Combining csvs from above.

In [198]:
files = glob("scraper_*")

In [199]:
print(files)

['scraper_0_999.csv', 'scraper_0_999.txt']


In [163]:
data = pd.DataFrame()

for file in files:
    temp_df = pd.read_csv(file, index_col=0)
    data = data.append(temp_df)

In [164]:
data

,library,Contributors,Dependencies,Dependent packages,Dependent repositories,First release,Forks,Latest release,Open Issues/PRs,Repository size,SourceRank,Stars,Total releases,Version Number,Version Time,Watchers
0,pymatchseries,1.0,10.0,0.0,0.0,"Oct 17, 2020",0.0,"Feb 5, 2021",0.0,2.17,7.0,3.0,2.0,"['0.1.0', '0.0.1']","[datetime.datetime(2021, 2, 5, 13, 21, tzinfo=...",1.0
1,reg,NaN,8.0,6.0,29.0,"Oct 28, 2013",NaN,"Jan 29, 2020",NaN,NaN,7.0,NaN,15.0,"['0.12', '0.11', '0.10', '0.9.3', '0.9.2', '0....","[datetime.datetime(2020, 1, 29, 22, 1, tzinfo=...",NaN
2,wadllib,NaN,0.0,2.0,443.0,"Mar 23, 2009",NaN,"Jan 20, 2021",NaN,NaN,9.0,NaN,5.0,"['1.3.5', '1.3.4', '1.3.3', '1.3.2', '0.1']","[datetime.datetime(2021, 1, 20, 11, 1, tzinfo=...",NaN
3,ipython.display,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,azure-mgmt-batchai,131.0,4.0,6.0,42.0,"Oct 3, 2017",1343.0,17 days ago,908.0,311.00,14.0,2001.0,6.0,"['7.0.0b1', '2.0.0', '1.0.1', '1.0.0', '0.2.0'...","[datetime.datetime(2021, 6, 3, 6, 30, tzinfo=<...",296.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1948,region,7.0,8.0,1.0,1.0,"May 8, 2018",16.0,"Jan 26, 2020",NaN,1.71,9.0,10.0,3.0,"['0.2.1', '0.2.0', '0.1.5']","[datetime.datetime(2020, 1, 26, 22, 31, tzinfo...",18.0
1949,cma,6.0,2.0,36.0,42.0,"May 12, 2014",102.0,"Apr 23, 2020",NaN,1.33,17.0,569.0,17.0,"['3.0.3', '3.0.2', '3.0.1', '3.0.0', '2.7.0', ...","[datetime.datetime(2020, 4, 23, 22, 52, tzinfo...",17.0
1950,causalgraphicalmodels,1.0,4.0,0.0,1.0,"Jul 1, 2018",0.0,"Sep 3, 2018",NaN,43.90,8.0,10.0,2.0,"['0.0.4', '0.0.3']","[datetime.datetime(2018, 9, 3, 17, 20, tzinfo=...",1.0
1951,napari,NaN,52.0,52.0,1.0,"Jul 30, 2018",NaN,about 24 hours ago,NaN,NaN,9.0,NaN,109.0,"['0.4.10', '0.4.10rc0', '0.4.9', '0.4.9rc3', '...","[datetime.datetime(2021, 6, 17, 18, 20, tzinfo...",NaN


In [165]:
data.to_csv("scraper_final.csv")